**Задача 1: Система управления библиотекой**

Создайте систему управления библиотекой используя collections:

1. Создайте namedtuple `Book` с полями: title, author, isbn, year
2. Создайте namedtuple `Reader` с полями: name, reader_id, phone
3. Используйте `defaultdict(list)` для хранения книг по жанрам
4. Используйте `deque` для очереди читателей, ожидающих популярную книгу
5. Используйте `Counter` для подсчета количества книг каждого автора
6. Используйте `OrderedDict` для хранения истории выдачи книг (читатель -> список книг)
7. Сериализуйте все данные в JSON и pickle форматы


In [1]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict
import json
import pickle

# 1) + 2)
Book = namedtuple('Book', ['title', 'author', 'isbn', 'year'])
Reader = namedtuple('Reader', ['name', 'reader_id', 'phone'])

# 3) Create and fill defaultdict with books
books_dict = defaultdict(list)
books_dict['scifi'].append(Book("fant1", "author1", "111", 2000))
books_dict['scifi'].append(Book("fant2", "author2", "222", 2000))
books_dict['drama'].append(Book("drama1", "author3", "333", 2000))
books_dict['drama'].append(Book("drama2", "author4", "444", 2000))
books_dict['action'].append(Book("act1", "author2", "555", 2000))
books_dict['action'].append(Book("act2", "author3", "666", 2000))

# 4) Create and fill deque with readers
readers : deque[namedtuple] = deque()
readers.append(Reader("name1", "11", "384"))
readers.append(Reader("name2", "22", "384"))
readers.append(Reader("name3", "33", "384"))
readers.append(Reader("name4", "44", "384"))

# 5) Counting authors
cnt = Counter()
for genre_books in books_dict.values():
    for book in genre_books:
        cnt[book.author] += 1

# 6)Create and fill OrderedDict
book_history : OrderedDict[namedtuple, namedtuple] = OrderedDict()
book_history[readers[0]] = books_dict["scifi"][0]
book_history[readers[1]] = books_dict["drama"][0]
book_history[readers[2]] = books_dict["action"][0]

# 7) Serialize into JSON
json_data = {
    "books_by_genre": {genre: [_._asdict() for _ in genre_books] for genre, genre_books in books_dict.items()},
    "readers_queue": [r._asdict() for r in readers],
    "author_counts": dict(cnt),
    "book_history": [
        {"reader": {"name": r.name, "reader_id": r.reader_id, "phone": r.phone},
         "book": b._asdict()}
        for r, b in book_history.items()
    ],
}

with open('library_data.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2)

# Serialize into pickle
with open('library_data.pkl', 'wb') as f:
    pickle.dump({
        "books_dict": books_dict,
        "readers": readers,
        "author_counts": cnt,
        "book_history": book_history,
    }, f)

In [ ]:
######################### ADDITIONAL TASK TO DO #########################
### Read and recreate saved file

# Read and recreate from JSON
with open('library_data.json', 'r', encoding='utf-8') as f:
    loaded_json = json.load(f)

# Recreate books_dict from JSON
books_dict_loaded = defaultdict(list)
for genre, books in loaded_json["books_by_genre"].items():
    books_dict_loaded[genre] = [Book(**b) for b in books]

# Recreate readers deque from JSON
readers_loaded = deque([Reader(**r) for r in loaded_json["readers_queue"]])

# Recreate author Counter from JSON
author_counts_loaded = Counter(loaded_json["author_counts"])

# Recreate book_history OrderedDict from JSON
book_history_loaded = OrderedDict()
for entry in loaded_json["book_history"]:
    reader = Reader(**entry["reader"])
    book = Book(**entry["book"])
    book_history_loaded[reader] = book

# Read and recreate from pickle
with open('library_data.pkl', 'rb') as f:
    pickle_data = pickle.load(f)

books_dict_from_pickle = pickle_data["books_dict"]
readers_from_pickle = pickle_data["readers"]
author_counts_from_pickle = pickle_data["author_counts"]
book_history_from_pickle = pickle_data["book_history"]

loaded_json, pickle_data

({'books_by_genre': {'scifi': [{'title': 'fant1',
     'author': 'author1',
     'isbn': '111',
     'year': 2000},
    {'title': 'fant2', 'author': 'author2', 'isbn': '222', 'year': 2000}],
   'drama': [{'title': 'drama1',
     'author': 'author3',
     'isbn': '333',
     'year': 2000},
    {'title': 'drama2', 'author': 'author4', 'isbn': '444', 'year': 2000}],
   'action': [{'title': 'act1',
     'author': 'author2',
     'isbn': '555',
     'year': 2000},
    {'title': 'act2', 'author': 'author3', 'isbn': '666', 'year': 2000}]},
  'readers_queue': [{'name': 'name1', 'reader_id': '11', 'phone': '384'},
   {'name': 'name2', 'reader_id': '22', 'phone': '384'},
   {'name': 'name3', 'reader_id': '33', 'phone': '384'},
   {'name': 'name4', 'reader_id': '44', 'phone': '384'}],
  'author_counts': {'author1': 1, 'author2': 2, 'author3': 2, 'author4': 1},
  'book_history': [{'reader': {'name': 'name1',
     'reader_id': '11',
     'phone': '384'},
    'book': {'title': 'fant1',
     'author'

**Задача 2: Анализатор файловой системы**

Создайте анализатор файловой системы используя os и sys:

1. Создайте namedtuple `FileInfo` с полями: name, size, extension, modified_time
2. Используйте `os.walk()` для обхода директории
3. Используйте `os.path` функции для получения информации о файлах
4. Используйте `Counter` для подсчета файлов по расширениям
5. Используйте `defaultdict(list)` для группировки файлов по размеру (маленькие < 1MB, средние 1-100MB, большие > 100MB)
6. Используйте `deque` для хранения последних 10 найденных файлов
7. Выведите статистику используя `sys.getsizeof()` для подсчета памяти
8. Сохраните результаты в JSON файл


In [ ]:
from collections import defaultdict, Counter, deque, namedtuple
import os
import sys
import json

# 0) Initialize data structures for 2), 5), 6)
extension_counter : Counter = Counter()
files_by_size : defaultdict = defaultdict(list)
last_10_files : deque = deque(maxlen=10)

# 1)
FileInfo : namedtuple = namedtuple('FileInfo', ['name', 'size', 'extension', 'modified_time'])

# 2) Scan directory with os.walk()
for root, dirs, files in os.walk("."):
    for filename in files:
        filepath = os.path.join(root, filename)
        
        # 3) Get file information (using os.path)
        file_size = os.path.getsize(filepath)
        file_ext = os.path.splitext(filename)[1].lower()
        mod_time = os.path.getmtime(filepath)
        
        # Create FileInfo object
        file_info = FileInfo(
            name=filename,
            size=file_size,
            extension=file_ext,
            modified_time=mod_time
        )
        
        # 4) Count files by extension (using Counter lib)
        extension_counter[file_ext] += 1

        # 5) Sort files by size (using defaultdict)
        if file_size < 1024 ** 2:   # < 1MB
            files_by_size['small'].append(file_info)
        elif file_size < 100 * 1024 * 1024:   # 1-100MB
            files_by_size['medium'].append(file_info)
        else:   # > 100MB
            files_by_size['large'].append(file_info)
        
        # 6) Add to deque for last 10 files
        last_10_files.append(file_info)

# 7) Calculate memory usage (using .getsizeof())
memory_usage = {
    'extension_counter': sys.getsizeof(extension_counter),
    'files_by_size': sys.getsizeof(files_by_size),
    'last_10_files': sys.getsizeof(last_10_files),
    'total': sys.getsizeof(extension_counter) + sys.getsizeof(files_by_size) + sys.getsizeof(last_10_files)
}

# Print statistics and memory usage (using .getsizeof())
print("Files by extension:", dict(extension_counter))
print("Files by size:", {k: len(v) for k, v in files_by_size.items()})
print("Last 10 files:", [f.name for f in last_10_files])
print("Memory usage:", memory_usage)

# 8) Serialize to JSON
json_data = {
    'files_by_extension': dict(extension_counter),
    'files_by_size_category': {
        category: [f._asdict() for f in files] 
        for category, files in files_by_size.items()
    },
    'last_10_files': [f._asdict() for f in last_10_files],
    'memory_usage': memory_usage
}

with open('filesystem_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2)

**Задача 3: Система конфигурации приложения**

Создайте систему конфигурации используя ChainMap и defaultdict:

1. Создайте namedtuple `Config` с полями: key, value, section, default_value
2. Создайте несколько словарей конфигурации (default, user, environment)
3. Используйте `ChainMap` для объединения конфигураций с приоритетом
4. Используйте `defaultdict(dict)` для группировки настроек по секциям
5. Используйте `OrderedDict` для сохранения порядка загрузки конфигураций
6. Используйте `os.environ` для чтения переменных окружения
7. Сериализуйте конфигурацию в JSON и pickle форматы

In [2]:
from collections import namedtuple, ChainMap, defaultdict, OrderedDict
import os
import json
import pickle
from typing import Dict, Any

from collections import namedtuple, ChainMap, defaultdict, OrderedDict
import os
from typing import Dict, Any

# 1) Config namedtuple
Config = namedtuple('Config', ['key', 'value', 'section', 'default_value'])

# 2) Configuration dictionaries
default_config = {
    'debug': False,
    'log_level': 'INFO',
    'timeout': 30,
}

user_config = {
    'log_level': 'DEBUG',
    'timeout': 60,
}

# Read from environment (only specific keys, as strings)
env_config = {}
if 'APP_DEBUG' in os.environ:
    env_config['debug'] = os.environ['APP_DEBUG']
if 'APP_LOG_LEVEL' in os.environ:
    env_config['log_level'] = os.environ['APP_LOG_LEVEL']

# 3) ChainMap: environment > user > default
config_chain = ChainMap(env_config, user_config, default_config)

# 4) Group by section
section_map = {
    'debug': 'app',
    'log_level': 'logging',
    'timeout': 'network',
}

grouped: defaultdict[str, Dict[str, Any]] = defaultdict(dict)
for key in config_chain:
    section = section_map.get(key, 'general')
    grouped[section][key] = config_chain[key]

# 5) OrderedDict for load order
config_order = OrderedDict([
    ('default', default_config),
    ('user', user_config),
    ('environment', env_config),
])

# Prepare Config objects
configs = [
    Config(
        key=key,
        value=config_chain[key],
        section=section_map.get(key, 'general'),
        default_value=default_config.get(key)
    )
    for key in config_chain
]

# 7) Serialize to JSON
json_data = {
    "active_config": dict(config_chain),
    "grouped_by_section": dict(grouped),
    "config_objects": [c._asdict() for c in configs],
    "load_order": dict(config_order),
}

with open('config.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2)

# Serialize to pickle
with open('config.pkl', 'wb') as f:
    pickle.dump({
        "chainmap": config_chain,
        "grouped": grouped,
        "order": config_order,
        "configs": configs,
    }, f)

**Задача 4: Мониторинг системы**

Создайте систему мониторинга используя sys и os:

1. Создайте namedtuple `SystemInfo` с полями: cpu_count, memory_usage, process_id, user_name
2. Используйте `os.cpu_count()` для получения количества процессоров
3. Используйте `sys.getallocatedblocks()` для мониторинга памяти
4. Используйте `os.getpid()` и `os.getlogin()` для информации о процессе
5. Используйте `deque` для хранения последних 20 измерений
6. Используйте `Counter` для подсчета частоты использования различных функций
7. Используйте `defaultdict(list)` для группировки измерений по времени
8. Сохраните историю мониторинга в pickle файл


**Задача 5: Система логирования**

Создайте систему логирования используя все изученные коллекции:

1. Создайте namedtuple `LogEntry` с полями: timestamp, level, message, module, function
2. Используйте `deque` для хранения последних 100 логов (кольцевой буфер)
3. Используйте `defaultdict(list)` для группировки логов по уровням (DEBUG, INFO, WARNING, ERROR)
4. Используйте `Counter` для подсчета количества логов каждого уровня
5. Используйте `OrderedDict` для хранения логов по времени (FIFO)
6. Используйте `ChainMap` для объединения различных источников логов
7. Используйте `os.path` для работы с файлами логов
8. Сериализуйте логи в JSON и pickle форматы


**Задача 6: Кэш-система**

Создайте простую кэш-систему используя collections:

1. Создайте namedtuple `CacheEntry` с полями: key, value, timestamp, access_count
2. Используйте `OrderedDict` для реализации LRU (Least Recently Used) кэша
3. Используйте `deque` для хранения истории доступа к ключам
4. Используйте `Counter` для подсчета частоты доступа к каждому ключу
5. Используйте `defaultdict(int)` для хранения счетчиков доступа
6. Реализуйте методы: get, set, delete, clear, size
7. Используйте `sys.getsizeof()` для мониторинга размера кэша
8. Сериализуйте кэш в pickle формат для сохранения между сессиями


**Задача 7: Анализатор текста**

Создайте анализатор текста используя collections:

1. Создайте namedtuple `WordInfo` с полями: word, frequency, length, first_occurrence
2. Используйте `Counter` для подсчета частоты слов
3. Используйте `defaultdict(list)` для группировки слов по длине
4. Используйте `deque` для хранения последних 50 уникальных слов
5. Используйте `OrderedDict` для хранения слов в порядке первого появления
6. Используйте `os.path` для работы с текстовыми файлами
7. Используйте `sys.getsizeof()` для анализа памяти
8. Сохраните результаты анализа в JSON файл


**Задача 8: Система управления задачами**

Создайте систему управления задачами (TODO) используя все изученные концепции:

1. Создайте namedtuple `Task` с полями: id, title, description, priority, status, created_date
2. Используйте `defaultdict(list)` для группировки задач по статусу (todo, in_progress, done)
3. Используйте `deque` для очереди задач с высоким приоритетом
4. Используйте `Counter` для подсчета задач по приоритету
5. Используйте `OrderedDict` для хранения задач в порядке создания
6. Используйте `ChainMap` для объединения различных списков задач
7. Используйте `os.path` для работы с файлами задач
8. Реализуйте функции: add_task, complete_task, get_tasks_by_status, get_priority_queue
9. Сериализуйте все данные в JSON и pickle форматы


**Задача 9: Система мониторинга производительности**

Создайте систему мониторинга производительности используя sys и collections:

1. Создайте namedtuple `PerformanceMetric` с полями: function_name, execution_time, memory_usage, timestamp
2. Используйте `deque` для хранения последних 100 измерений производительности
3. Используйте `defaultdict(list)` для группировки метрик по функциям
4. Используйте `Counter` для подсчета количества вызовов каждой функции
5. Используйте `OrderedDict` для хранения метрик в хронологическом порядке
6. Используйте `sys.getsizeof()` для мониторинга памяти
7. Используйте `os.path` для работы с файлами метрик
8. Реализуйте функции: record_metric, get_function_stats, get_memory_usage, export_metrics
9. Сериализуйте метрики в JSON и pickle форматы


**Задача 10: Комплексная система управления данными**

Создайте комплексную систему управления данными, объединяющую все изученные концепции:

1. Создайте несколько namedtuple для различных типов данных (User, Product, Order, etc.)
2. Используйте `defaultdict` для создания индексов по различным полям
3. Используйте `deque` для реализации очередей обработки данных
4. Используйте `Counter` для аналитики и статистики
5. Используйте `OrderedDict` для хранения данных в определенном порядке
6. Используйте `ChainMap` для объединения различных источников данных
7. Используйте `os` и `sys` для работы с файловой системой и мониторинга
8. Реализуйте CRUD операции (Create, Read, Update, Delete)
9. Добавьте функции экспорта/импорта данных в различных форматах
10. Сериализуйте все данные в JSON, pickle и другие форматы
11. Добавьте типизацию для всех функций и классов
12. Реализуйте систему логирования для отслеживания операций
